In [8]:
%%capture --no-stderr --no-display
!pip install langgraph langchain_community langchain_openai llama_index llama-index-llms-groq

In [4]:
import os
os.environ["OPENAI_API_KEY"] = ''
groq_api_key = os.environ["GROQ_API_KEY"] = ''

In [9]:
from llama_index.llms.groq import Groq

llama3 = Groq(model="llama3-8b-8192", api_key=groq_api_key, temperature=0.0)

llama3

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d4d40336c80>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7d4d112575b0>, completion_to_prompt=<function default_completion_to_prompt at 0x7d4d110ebf40>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama3-8b-8192', temperature=0.0, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [3]:
from typing import Optional, List
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Class Representing Individual Person."""

    name: str = Field(description="Name of the person.")
    age: int = Field(description="Age of Person.")
    height: Optional[str] = Field(description="Height of Person")

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]

## Generate Structured Output

In [10]:
from llama_index.core.program import LLMTextCompletionProgram, FunctionCallingProgram
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
You are an expert data parser. Parse data from user query.

If you don't know any field then set it to None.

{query}
"""

pydantic_data_parser = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Person),
    #output_cls=Person,
    prompt_template_str=prompt_template_str,
    llm=llama3
)

In [11]:
response = pydantic_data_parser(query="Anna is 20 years old and five foot five inch.")

response

Person(name='Anna', age=20, height='five foot five inch')